In [1]:
%matplotlib inline

import gensim
import logging
import math
import nltk
import os
import pyLDAvis.gensim_models
import random
import re
import spacy
import json
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from matplotlib import cm

from imageio import imread
from wordcloud import WordCloud

from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.models.ldamodel import LdaModel

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')
from nltk.corpus import stopwords # est-ce que c'est le même de sklearn ?
from nltk import pos_tag, ne_chunk
from nltk.tokenize import word_tokenize
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.collocations import TrigramCollocationFinder
from nltk.metrics import TrigramAssocMeasures
from nltk.tokenize import RegexpTokenizer

C:\Users\brune\anaconda3\envs\CoursHN\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
C:\Users\brune\anaconda3\envs\CoursHN\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\brune\anaconda3\envs\CoursHN\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
C:\Users\brune\anaconda3\envs\CoursHN\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:36: Depr

C:\Users\brune\anaconda3\envs\CoursHN\lib\site-packages\keras\utils\image_utils.py:36: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
C:\Users\brune\anaconda3\envs\CoursHN\lib\site-packages\keras\utils\image_utils.py:37: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': pil_image.BILINEAR,
C:\Users\brune\anaconda3\envs\CoursHN\lib\site-packages\keras\utils\image_utils.py:38: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  'bicubic': pil_image.BICUBIC,
C:\Users\brune\anaconda3\envs\CoursHN\lib\site-packages\keras\utils\image_utils.py:39: DeprecationWarning: HAMMING is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.HAMMING instead.
  'hamming': pil_image.HAMMING,
C:\Users\brune\anacon

In [2]:
with open(os.path.join('posts_maior_propre.json'), 'r', encoding = 'UTF-8') as fin_M2 :
    data = json.load(fin_M2)

In [3]:
type(data)

list

In [4]:
len(data)

110895

In [5]:
data[9377]

"LE PARiSIEN de vendredi : ...........BERTRAND PICCARD , je retranscris ........Oui! pour celles , ceux qui ONT le TEMPS , ou pour des chose priVées sans CONTRAINTES de temsps ( c.a.d. : ceux , celles qui VOyAGENT pour leur travail !.......ou les LONGUES diStances ...: exemple ce CHINE - NICE : ......, Saïgon , HaNoÏ - paris , ou L.A. :................, Paris -TEHRAN etc !........trains de marchndises existent entre CHINE et FRANCE - : pas pour des produits frais , NON ? ......) 36 heures DANS pour de suède rejoindre PARIS . Je ne suis pas CLIMATOsceptique mais JE persiste : La 'tite JEANNE d'ARC de notre combat contre Les EARTH 's KILLERS , VA trop LOIN !........: train voilier !......les psuedo' expériences scientifuqes pour JUSTiFIER avec ou sans sponsor ....leur little help pour la CAUSE aurait des RETomTéEs poSItiVES pour eux ............Je me rappelle TOM et JERRY ( les glaces à , AUCHAN , etc ...ma soeur de Londres , ...Les BILL GATES , buffet , SOROS etc !........../ donnons et

In [6]:
index = ''
a = ''
list_clean_content = []

for j in range(len(data)) :
    index = data[j]
    a = index.replace( "\n", "")
    list_clean_content.append(a)

In [7]:
list_clean_content[1377]

"Marchant dans Londres, ville qu’elle connaît bien et qu’elle sillonne depuis de nombreuses années, Sylvie Doizelet a pris, pendant ses récents séjours, des séries de photos qui disent les dangers, interdictions, dérapages, risques de chute, de contamination et de mort qui guettent l’imprudent qui oublierait de prendre au sérieux tous ces rappels.Ils fleurissent sur de multiples panneaux. On les trouve partout : des terrains vagues au cœur de la ville en passant par les trottoirs, les grilles, les cages d’escalier, les façades de verre, les quais, les esplanades, les souterrains… Façon pour les uns de se prémunir et pour les autres de trembloter en se méfiant de ces incessants périls susceptibles de transformer les éventuelles (et de plus en plus problématiques) flâneries en guet-apens.Ces dangers inhérents à la capitale londonienne, Jean-Claude Pirotte a choisi de les contourner sans s’en laisser conter. Ces pancartes vont, au contraire, l’inciter à se promener en convoquant, dans les

In [8]:
'''from stop_words import get_stop_words
fr_stop_words = get_stop_words('french')
fr_stop_words'''

"from stop_words import get_stop_words\nfr_stop_words = get_stop_words('french')\nfr_stop_words"

In [9]:
def sent_to_words(sentences) :
    for sentence in sentences :
        yield(simple_preprocess(str(sentence), deacc = True))

In [10]:
doc_set = list_clean_content
data_words = list(sent_to_words(doc_set))

In [11]:
ndocs = len(data_words)
print(ndocs)

110895


In [12]:
with open("Stop-words-french.txt", "r", encoding='utf8') as f:
    stop_words = [line.rstrip("\n") for line in f.readlines()]
    
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# on retire les mots-outils
data_words_nostops = remove_stopwords(data_words)

In [13]:
# création du dictionnaire
dico = corpora.Dictionary(data_words_nostops)

# ce qui permet par ex. de filtrer le vocabulaire
dico.filter_extremes(no_below = 10)

# Create Corpus
texts = data_words_nostops

# matrice Term Document Frequency
corpus = [dico.doc2bow(text) for text in texts]

In [14]:
len(dico)

70469

In [15]:
ntopics = 10

In [16]:
generate_lda = True
#generate_lda = False

# generate LDA model
import logging

if generate_lda :
    print("generate new LDA model")
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level = logging.INFO)
    ldamodel = LdaModel( corpus, 
                         num_topics = ntopics, 
                         id2word = dico,
                         passes = 2, 
                         random_state = 100, 
                         per_word_topics = True)
    
    print(ldamodel)

2023-01-23 10:59:50,485 : INFO : using symmetric alpha at 0.1
2023-01-23 10:59:50,485 : INFO : using symmetric eta at 0.1
2023-01-23 10:59:50,485 : INFO : using serial LDA version on this node
2023-01-23 10:59:50,549 : INFO : running online (multi-pass) LDA training, 10 topics, 2 passes over the supplied corpus of 110895 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2023-01-23 10:59:50,549 : INFO : PROGRESS: pass 0, at document #2000/110895


generate new LDA model


2023-01-23 10:59:51,507 : INFO : merging changes from 2000 documents into a model of 110895 documents
2023-01-23 10:59:51,539 : INFO : topic #8 (0.100): 0.006*"entre" + 0.004*"ai" + 0.003*"celui" + 0.003*"monde" + 0.003*"ont" + 0.003*"vie" + 0.003*"jour" + 0.003*"quelques" + 0.002*"corps" + 0.002*"livre"
2023-01-23 10:59:51,539 : INFO : topic #5 (0.100): 0.004*"quelques" + 0.004*"ont" + 0.004*"vie" + 0.003*"livre" + 0.003*"ai" + 0.003*"celui" + 0.003*"homme" + 0.003*"editions" + 0.002*"entre" + 0.002*"mort"
2023-01-23 10:59:51,555 : INFO : topic #4 (0.100): 0.004*"entre" + 0.004*"ai" + 0.004*"vie" + 0.003*"quelques" + 0.003*"ont" + 0.003*"monde" + 0.003*"livre" + 0.002*"celui" + 0.002*"jour" + 0.002*"jean"
2023-01-23 10:59:51,555 : INFO : topic #0 (0.100): 0.005*"entre" + 0.004*"vie" + 0.004*"ai" + 0.003*"ont" + 0.003*"livre" + 0.003*"moi" + 0.003*"monde" + 0.003*"celui" + 0.003*"corps" + 0.003*"editions"
2023-01-23 10:59:51,555 : INFO : topic #2 (0.100): 0.005*"entre" + 0.004*"vie" + 

2023-01-23 10:59:57,874 : INFO : topic #8 (0.100): 0.007*"ai" + 0.007*"songs" + 0.005*"amour" + 0.005*"song" + 0.004*"dalida" + 0.004*"te" + 0.004*"vie" + 0.004*"ans" + 0.004*"entre" + 0.004*"monde"
2023-01-23 10:59:57,874 : INFO : topic #7 (0.100): 0.018*"songs" + 0.007*"youtube" + 0.006*"song" + 0.006*"ai" + 0.005*"ans" + 0.005*"fils" + 0.004*"vie" + 0.003*"bobby" + 0.003*"papa" + 0.003*"film"
2023-01-23 10:59:57,874 : INFO : topic diff=0.360891, rho=0.377964
2023-01-23 10:59:57,874 : INFO : PROGRESS: pass 0, at document #16000/110895
2023-01-23 10:59:58,613 : INFO : merging changes from 2000 documents into a model of 110895 documents
2023-01-23 10:59:58,645 : INFO : topic #4 (0.100): 0.025*"ai" + 0.010*"michel" + 0.005*"quelques" + 0.004*"idees" + 0.004*"volkovitch" + 0.003*"trouve" + 0.003*"ont" + 0.003*"entre" + 0.003*"vie" + 0.003*"monde"
2023-01-23 10:59:58,645 : INFO : topic #8 (0.100): 0.007*"ai" + 0.005*"amour" + 0.005*"songs" + 0.005*"te" + 0.004*"monde" + 0.004*"song" + 0.0

2023-01-23 11:00:06,035 : INFO : merging changes from 2000 documents into a model of 110895 documents
2023-01-23 11:00:06,066 : INFO : topic #9 (0.100): 0.005*"monde" + 0.005*"contre" + 0.004*"ont" + 0.004*"ai" + 0.004*"guerre" + 0.003*"ans" + 0.003*"photos" + 0.003*"pays" + 0.003*"blogs" + 0.003*"entre"
2023-01-23 11:00:06,066 : INFO : topic #1 (0.100): 0.006*"livre" + 0.005*"france" + 0.004*"vie" + 0.004*"ont" + 0.003*"livres" + 0.003*"avignon" + 0.003*"chez" + 0.003*"pays" + 0.003*"monde" + 0.003*"histoire"
2023-01-23 11:00:06,066 : INFO : topic #4 (0.100): 0.018*"ai" + 0.008*"blogspot" + 0.006*"article" + 0.005*"quelques" + 0.005*"texte" + 0.005*"entre" + 0.005*"html" + 0.004*"textes" + 0.004*"communicants" + 0.003*"brigetoun"
2023-01-23 11:00:06,066 : INFO : topic #6 (0.100): 0.008*"ai" + 0.004*"ans" + 0.004*"jean" + 0.004*"aime" + 0.004*"film" + 0.003*"rock" + 0.003*"songs" + 0.003*"livre" + 0.003*"love" + 0.003*"paris"
2023-01-23 11:00:06,066 : INFO : topic #3 (0.100): 0.018*"ai

...

2023-01-23 11:01:09,722 : INFO : topic #9 (0.100): 0.009*"ont" + 0.007*"monde" + 0.006*"nos" + 0.005*"contre" + 0.005*"petition" + 0.005*"pays" + 0.003*"france" + 0.003*"avons" + 0.003*"droits" + 0.003*"toute"
2023-01-23 11:01:09,738 : INFO : topic diff=0.107042, rho=0.131936
2023-01-23 11:01:09,738 : INFO : PROGRESS: pass 1, at document #70000/110895
2023-01-23 11:01:10,245 : INFO : merging changes from 2000 documents into a model of 110895 documents
2023-01-23 11:01:10,274 : INFO : topic #9 (0.100): 0.009*"ont" + 0.008*"monde" + 0.006*"nos" + 0.005*"contre" + 0.005*"pays" + 0.004*"petition" + 0.003*"france" + 0.003*"droits" + 0.003*"avons" + 0.003*"toute"
2023-01-23 11:01:10,274 : INFO : topic #5 (0.100): 0.019*"poesie" + 0.019*"revue" + 0.016*"jean" + 0.014*"ral" + 0.013*"php" + 0.011*"poete" + 0.010*"poemes" + 0.009*"poeme" + 0.008*"pierre" + 0.007*"patrick"
2023-01-23 11:01:10,289 : INFO : topic #7 (0.100): 0.017*"musique" + 0.016*"film" + 0.008*"laterit" + 0.007*"festival" + 0.00

2023-01-23 11:01:15,403 : INFO : merging changes from 2000 documents into a model of 110895 documents
2023-01-23 11:01:15,450 : INFO : topic #9 (0.100): 0.009*"ont" + 0.008*"monde" + 0.006*"nos" + 0.005*"contre" + 0.004*"pays" + 0.004*"vote" + 0.003*"petition" + 0.003*"avons" + 0.003*"toute" + 0.003*"hui"
2023-01-23 11:01:15,450 : INFO : topic #2 (0.100): 0.010*"ville" + 0.009*"jour" + 0.008*"ciel" + 0.008*"eau" + 0.007*"nuit" + 0.006*"soleil" + 0.006*"vent" + 0.005*"entre" + 0.005*"air" + 0.005*"loin"
2023-01-23 11:01:15,452 : INFO : topic #3 (0.100): 0.023*"ai" + 0.014*"aime" + 0.014*"apprecierez" + 0.012*"avez" + 0.010*"post" + 0.009*"jour" + 0.006*"pis" + 0.006*"ans" + 0.006*"chez" + 0.004*"fille"
2023-01-23 11:01:15,452 : INFO : topic #7 (0.100): 0.015*"musique" + 0.015*"film" + 0.010*"public" + 0.009*"tournoi" + 0.008*"scene" + 0.007*"duel" + 0.007*"finale" + 0.006*"cinema" + 0.006*"voix" + 0.005*"theatre"
2023-01-23 11:01:15,452 : INFO : topic #8 (0.100): 0.014*"corps" + 0.011*"

2023-01-23 11:01:18,422 : INFO : topic #7 (0.100): 0.015*"film" + 0.014*"musique" + 0.009*"scene" + 0.007*"public" + 0.006*"cinema" + 0.006*"theatre" + 0.006*"tournoi" + 0.005*"voix" + 0.005*"festival" + 0.004*"finale"
2023-01-23 11:01:18,422 : INFO : topic #9 (0.100): 0.010*"ont" + 0.008*"monde" + 0.006*"nos" + 0.005*"pays" + 0.005*"contre" + 0.003*"toute" + 0.003*"entre" + 0.003*"vie" + 0.003*"hui" + 0.003*"aujourd"
2023-01-23 11:01:18,422 : INFO : topic #3 (0.100): 0.020*"ai" + 0.016*"saison" + 0.015*"prenoms" + 0.009*"aime" + 0.008*"chez" + 0.008*"jour" + 0.008*"avez" + 0.007*"apprecierez" + 0.006*"post" + 0.005*"mercredi"
2023-01-23 11:01:18,422 : INFO : topic diff=0.064106, rho=0.131936
2023-01-23 11:01:18,422 : INFO : PROGRESS: pass 1, at document #96000/110895
2023-01-23 11:01:19,193 : INFO : merging changes from 2000 documents into a model of 110895 documents
2023-01-23 11:01:19,225 : INFO : topic #0 (0.100): 0.016*"ai" + 0.009*"moi" + 0.005*"vie" + 0.004*"avais" + 0.004*"sais

2023-01-23 11:01:23,550 : INFO : topic #8 (0.100): 0.012*"corps" + 0.012*"nos" + 0.008*"nuit" + 0.008*"mots" + 0.008*"monde" + 0.007*"amour" + 0.007*"voix" + 0.007*"silence" + 0.007*"yeux" + 0.006*"homme"
2023-01-23 11:01:23,550 : INFO : topic diff=0.140006, rho=0.131936
2023-01-23 11:01:23,550 : INFO : PROGRESS: pass 1, at document #108000/110895
2023-01-23 11:01:24,148 : INFO : merging changes from 2000 documents into a model of 110895 documents
2023-01-23 11:01:24,180 : INFO : topic #2 (0.100): 0.009*"ville" + 0.008*"eau" + 0.007*"ciel" + 0.007*"nuit" + 0.006*"jour" + 0.006*"soleil" + 0.005*"vent" + 0.005*"entre" + 0.004*"pluie" + 0.004*"mer"
2023-01-23 11:01:24,180 : INFO : topic #7 (0.100): 0.019*"theatre" + 0.018*"scene" + 0.012*"musique" + 0.011*"film" + 0.008*"spectacle" + 0.006*"comediens" + 0.006*"public" + 0.006*"voix" + 0.006*"festival" + 0.006*"piece"
2023-01-23 11:01:24,180 : INFO : topic #0 (0.100): 0.018*"ai" + 0.011*"moi" + 0.005*"vie" + 0.004*"sais" + 0.004*"monde" + 

LdaModel(num_terms=70469, num_topics=10, decay=0.5, chunksize=2000)


In [17]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare( ldamodel,
                                      gensim.matutils.corpus2csc(corpus),
                                      dictionary = ldamodel.id2word)

pyLDAvis.display(vis)

C:\Users\brune\anaconda3\envs\CoursHN\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)
